# 04 tiering archetypes Notebook

This notebook implements the analysis for the 04 tiering archetypes stage of the Fantasy Football Analysis project.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the src directory to path
sys.path.append('..')

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [ ]:
# TODO: Implement 04 tiering archetypes analysis

In [ ]:
# Import project modules
from src.data.data_loader import load_config, load_all_data
from src.data.data_processor import load_csv_data
from src.analysis.tiering import (
    identify_player_tiers,
    identify_player_archetypes,
    analyze_tiers_vs_expectations
)
from src.visualization.tiering_vis import (
    plot_player_tiers,
    plot_player_archetypes,
    plot_tier_expectations
)
from src.utils.validation import validate_analysis_output

# Set pandas display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [ ]:
# 1. Load configuration and processed data
config = load_config()
vorp_df = load_csv_data(os.path.join(config['data_paths']['processed_data'], 'vorp_analysis.csv'))


In [ ]:
# 2. Identify performance tiers
tier_results = identify_player_tiers(vorp_df)

# Display tier stats for each position
positions = ['QB', 'RB', 'WR', 'TE']
for pos in positions:
    tier_stats_key = f"{pos}_tier_stats"
    if tier_stats_key in tier_results:
        print(f"\n{pos} Performance Tiers:")
        display(tier_results[tier_stats_key])

In [ ]:
# 3. Identify player archetypes
# Define features for each position
position_features = {
    'QB': ['Pass_Points_Share', 'Rush_Points_Share', 'Pass_Yards_Per_Att', 'TD_Rate', 'Int_Rate'],
    'RB': ['Rush_Points_Share', 'Recv_Points_Share', 'Rush_Yards_Per_Att', 'Catch_Rate'],
    'WR': ['Yards_Per_Reception', 'Catch_Rate', 'TD_Rate', 'YPRR', 'aDOT'],
    'TE': ['Yards_Per_Reception', 'Catch_Rate', 'TD_Rate', 'YPRR', 'Route_Rate', 'Inline_Rate']
}

archetype_results = identify_player_archetypes(vorp_df, position_features)

# Display archetype stats for each position
for pos in positions:
    archetype_stats_key = f"{pos}_archetype_stats"
    feature_key = f"{pos}_archetype_features"
    
    if archetype_stats_key in archetype_results:
        print(f"\n{pos} Archetypes:")
        display(archetype_results[archetype_stats_key])
        
        if feature_key in archetype_results:
            print(f"\n{pos} Archetype Feature Means:")
            display(archetype_results[feature_key])


In [ ]:
# 4. Analyze tiers vs expectations
tier_expectations = analyze_tiers_vs_expectations(tier_results, vorp_df)

# Display tier expectations analysis
for pos in positions:
    expectations_key = f"{pos}_tier_expectations"
    if expectations_key in tier_expectations:
        print(f"\n{pos} Tier Expectations Analysis:")
        display(tier_expectations[expectations_key])

In [ ]:
# 5. Create visualizations
# Plot player tiers
for pos in positions:
    tier_key = f"{pos}_tiers"
    if tier_key in tier_results:
        plt.figure(figsize=(12, 8))
        plot_player_tiers(tier_results[tier_key], pos)
        plt.tight_layout()
        plt.savefig(f'../outputs/figures/{pos}_tiers.png')
        plt.show()

# Plot player archetypes
for pos in positions:
    archetype_key = f"{pos}_archetypes"
    if archetype_key in archetype_results:
        plt.figure(figsize=(12, 8))
        plot_player_archetypes(archetype_results[archetype_key], pos, position_features[pos])
        plt.tight_layout()
        plt.savefig(f'../outputs/figures/{pos}_archetypes.png')
        plt.show()

# Plot tier expectations
if 'all_tier_expectations' in tier_expectations:
    plt.figure(figsize=(14, 10))
    plot_tier_expectations(tier_expectations['all_tier_expectations'])
    plt.tight_layout()
    plt.savefig('../outputs/figures/tier_expectations.png')
    plt.show()

In [ ]:
# 6. Save processed data
for key, df in tier_results.items():
    df.to_csv(os.path.join(config['data_paths']['processed_data'], f"{key}.csv"), index=False)

for key, df in archetype_results.items():
    df.to_csv(os.path.join(config['data_paths']['processed_data'], f"{key}.csv"), index=False)

for key, df in tier_expectations.items():
    df.to_csv(os.path.join(config['data_paths']['processed_data'], f"{key}.csv"), index=False)

print("Tiering and archetype analysis completed!")